In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time
import collections
import math
import random
import zipfile
import fileUtil
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
import csv
import jieba.posseg as jp, jieba

from gensim.models import doc2vec, ldamodel
from gensim import corpora, models

In [2]:
dir_name = 'dataset'
file_name = 'hotel_desc_seed_remove_200line.csv'
document = fileUtil.read_data_as_sentence_array(dir_name, file_name)
# 文本长度是指文本中的字符个数
print ('Length of text: {} characters'.format(len(document)))

段落处理完毕
Length of text: 100 characters


In [3]:
print(document[:5])

['上海时生隅园林文化酒店 2017年的秋末 时生隅初见END彼时的心愿 想要在快节奏的都市 寻一处讲诉时光和生命的角落END不问对错 不闻浮沉END这便是时生隅的由来END这一次 我们秉承初心 在闹中取静的川沙地铁站商圈 觅得一方天地 充满江南气息的园林 叶园END长廊湖景 古雅庭院END这里淡雅而大气 是快节奏与民俗文化的交融 这里幽静而明快 是远道而来亦或小憩而居的别样天地END时生隅的新老朋友 我们在叶园 期待与您共襄当下景 把时光言欢END', '云和夜泊酒店 位于浦东新区祝桥秋亭路 毗邻上海浦东国际机场 车程20分钟 距上海迪士尼18分钟车程 距上海野生动物园21分钟车程END酒店地理位置优越 并配备大巴 中巴 豪华轿车接送客户END酒店拥有房间98间 大床30间 行政大床15间 标间30间 亲子房23间END', '维也纳酒店位于浦东新区川南奉公路 近晨阳路 与迪士尼直线距离约8公里 可便捷到达地铁2号线凌空路站 交通便利END酒店周围生活设施齐全 旅游资源众多 有上海新国际博览中心 迪士尼 上海野生动物园 农艺大观园 三甲港滨海旅游区等END维也纳酒店是维也纳旗下的连锁酒店 装修豪华舒适 整体风格高贵典雅END客房宽敞明亮 温馨时尚 房内布置精美 处处体现人性化的理念END酒店还有宽敞停车场 休闲茶吧 宽敞会议室等 同时还为您提供精品早餐 浦东机场接机等服务 是商务 休闲 会务的理想酒店END', '上海万信r酒店位于浦东新区崮山路 地处内环线陆家嘴金融区内 近地铁6号线 地铁9号线 配备专属停车场 自驾至新国际博览中心约10分钟 至外滩 世博园区 南京路步行街 上海自由贸易区 上海迪斯尼乐园约20分钟END酒店拥有百余间崭新客房 精选配套 让舒适与轻奢相拥 酒店圣罗拉餐厅环境舒适安逸 主打西餐 辅以粤菜 川菜 沪菜 在此享受由港粤名厨精心烹饪的佳肴 舌尖不由自主的跳起了芭蕾END早餐的菜品琳琅满目可达40种 为保证口味 餐品均选优质食材当天加工END酒店豪华宴会厅提供一站式婚礼服务与专业的会务服务END万信厅可支持约120人课桌式会议 丽都厅适合60人以下的各类会议需求END万信r酒店设计 突破了万信酒店一贯理念 时尚年轻 秉承r文化 华丽 贴心 轻松 享受END', '酒店位于浦东新区沪南公路3655弄2号 素有小上海之称的周浦镇 沪南公路横桥路

In [4]:
flags = ('n', 'f', 's', 'nr', 'ns', 'nt', 'nw', 'nz','v','vd','vn','a','ad','an','d','q','r','p','c','u','xc','PER','LOC','ORG')  # 词性

words_ls = []
for text in document:
    temp_words = [w.word for w in jp.cut(text,use_paddle=True)]
    if len(temp_words) > 0:
        words_ls.extend(temp_words)
# print('++++++++{}', words_ls[:40])

# cut_words = [w.word for w in jp.cut(document,use_paddle=True)]
# cut_words = [w.word for w in jp.cut(document,use_paddle=True) if w.flag in flags and len(w.word) > 0]
# print(cut_words)
# for hotelDesc in document:
#     temp_words = [w.word for w in jp.cut(hotelDesc,use_paddle=True) if w.flag in flags and w.word not in stopWords and len(w.word) > 0]
#     temp_speechs = [w.flag for w in jp.cut(hotelDesc,use_paddle=True) if w.flag in flags and w.word not in stopWords and len(w.word) > 0]
# #     words = [w.word for w in jp.cut(text) if w.flag in flags and w.word not in stopwords and len(w.word) > 0]
# #     print('---{}, {}',temp_words, len(temp_words))
# #     print('---{}, {}',temp_speechs, len(temp_speechs))
#     if len(temp_words) > 0:
#         #[[句一分词列表],[句二分词列表],...[句五分词列表]]
#         words_ls.append(temp_words)
#         #[[句一词性列表],[句二词性列表],...[句五词性列表]]
#         words_speech.append(temp_speechs)
#         #五句合并的一个列表
#         whole_words.extend(temp_words)
# print(words_ls)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.675 seconds.
Prefix dict has been built successfully.


In [5]:
vocab = sorted(set(words_ls))
print(vocab[:100])
print ('{} unique characters'.format(len(vocab)))

[' ', '!', '+', '-', '00', '000', '01', '1', '10', '100', '1088', '11', '1132', '12', '120', '1200', '12000', '1245', '13', '135', '1389', '14', '140', '15', '150', '156', '16', '17', '173', '1788', '18', '180', '1856', '19', '1930', '2', '20', '200', '2017', '21', '211', '228', '23', '24', '25', '250', '3', '30', '300', '304', '31', '3655', '37', '4', '40', '42', '45', '450', '4s店', '5', '50', '500', '5000', '55', '6', '60', '63', '65', '66', '7', '70', '700', '71', '736', '8', '800', '82', '836', '868', '89', '9', '90', '920', '930', '944', '958', '96', '98', '985', 'END', 'END2', 'END2012', 'END2015', 'END2018', 'END24', 'END5', 'END55', 'END5km', 'END720', 'END8']
3173 unique characters


In [6]:
text2 = open(os.path.join(dir_name,file_name), 'rb').read().decode(encoding='utf-8')

# 文本长度是指文本中的字符个数
print ('Length of text: {} characters'.format(len(text2)))

Length of text: 30324 characters


In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in words_ls])
print(text_as_int)

[ 243 1771 1033 ...  678 2414   89]


In [8]:
print(idx2char)

[' ' '!' '+' ... '龙华' '龙柏' '龙阳路']


In [9]:
print(text_as_int)

[ 243 1771 1033 ...  678 2414   89]


In [10]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '!' :   1,
  '+' :   2,
  '-' :   3,
  '00':   4,
  '000':   5,
  '01':   6,
  '1' :   7,
  '10':   8,
  '100':   9,
  '1088':  10,
  '11':  11,
  '1132':  12,
  '12':  13,
  '120':  14,
  '1200':  15,
  '12000':  16,
  '1245':  17,
  '13':  18,
  '135':  19,
  ...
}


In [11]:
# 设定每个输入句子长度的最大值
seq_length = 100
examples_per_epoch = len(text2)//seq_length

# 创建训练样本 / 目标
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

上海
时生隅
园林
文化
酒店


In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'上海时生隅园林文化酒店 2017年的秋末 时生隅初见END彼时的心愿 想要在快节奏的都市 寻一处讲诉时光和生命的角落END不问对错 不闻浮沉END这便是时生隅的由来END这一次 我们秉承初心 在闹中取静的川沙地铁站商圈 觅得一方天地 充满江南气息的园林 叶园END长廊湖景 古雅庭院END这里淡雅而大气 是快节奏与民俗文化的交融 这里幽静而明快 是远道而来亦或小憩'
'而居的别样天地END时生隅的新老朋友 我们在叶园 期待与您共襄当下景 把时光言欢END云和夜泊酒店 位于浦东新区祝桥秋亭路 毗邻上海浦东国际机场 车程20分钟 距上海迪士尼18分钟车程 距上海野生动物园21分钟车程END酒店地理位置优越 并配备大巴 中巴 豪华轿车接送客户END酒店拥有房间98间 大床30间 行政大床15间 标间30间 亲子房23间END维也纳酒店位于浦东新区川南'
'奉公路 近晨阳路 与迪士尼直线距离约8公里 可便捷到达地铁2号线凌空路站 交通便利END酒店周围生活设施齐全 旅游资源众多 有上海新国际博览中心 迪士尼 上海野生动物园 农艺大观园 三甲港滨海旅游区等END维也纳酒店是维也纳旗下的连锁酒店 装修豪华舒适 整体风格高贵典雅END客房宽敞明亮 温馨时尚 房内布置精美 处处体现人性化的理念END酒店还有宽敞停车场 休闲茶吧 宽敞'
'会议室等 同时还为您提供精品早餐 浦东机场接机等服务 是商务 休闲 会务的理想酒店END上海万信r酒店位于浦东新区崮山路 地处内环线陆家嘴金融区内 近地铁6号线 地铁9号线 配备专属停车场 自驾至新国际博览中心约10分钟 至外滩 世博园区 南京路步行街 上海自由贸易区 上海迪斯尼乐园约20分钟END酒店拥有百余间崭新客房 精选配套 让舒适与轻奢相拥 酒店圣罗拉餐厅环境'
'舒适安逸 主打西餐 辅以粤菜 川菜 沪菜 在此享受由港粤名厨精心烹饪的佳肴 舌尖不由自主的跳起了芭蕾END早餐的菜品琳琅满目可达40种 为保证口味 餐品均选优质食材当天加工END酒店豪华宴会厅提供一站式婚礼服务与专业的会务服务END万信厅可支持约120人课桌式会议 丽都厅适合60人以下的各类会议需求END万信r酒店设计 突破了万信酒店一贯理念 时尚年轻 '


In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '上海时生隅园林文化酒店 2017年的秋末 时生隅初见END彼时的心愿 想要在快节奏的都市 寻一处讲诉时光和生命的角落END不问对错 不闻浮沉END这便是时生隅的由来END这一次 我们秉承初心 在闹中取静的川沙地铁站商圈 觅得一方天地 充满江南气息的园林 叶园END长廊湖景 古雅庭院END这里淡雅而大气 是快节奏与民俗文化的交融 这里幽静而明快 是远道而来亦或'
Target data: '时生隅园林文化酒店 2017年的秋末 时生隅初见END彼时的心愿 想要在快节奏的都市 寻一处讲诉时光和生命的角落END不问对错 不闻浮沉END这便是时生隅的由来END这一次 我们秉承初心 在闹中取静的川沙地铁站商圈 觅得一方天地 充满江南气息的园林 叶园END长廊湖景 古雅庭院END这里淡雅而大气 是快节奏与民俗文化的交融 这里幽静而明快 是远道而来亦或小憩'


In [15]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 243 ('上海')
  expected output: 1771 ('时生隅')
Step    1
  input: 1771 ('时生隅')
  expected output: 1033 ('园林')
Step    2
  input: 1033 ('园林')
  expected output: 1697 ('文化')
Step    3
  input: 1697 ('文化')
  expected output: 2918 ('酒店')
Step    4
  input: 2918 ('酒店')
  expected output: 0 (' ')


In [16]:
# 批大小
BATCH_SIZE = 128

# 设定缓冲区大小，以重新排列数据集
# （TF 数据被设计为可以处理可能是无限的序列，
# 所以它不会试图在内存中重新排列整个序列。相反，
# 它维持一个缓冲区，在缓冲区重新排列元素。） 
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((128, 100), (128, 100)), types: (tf.int64, tf.int64)>

In [17]:
# 词集的长度
vocab_size = len(vocab)

# 嵌入的维度
embedding_dim = 256

# RNN 的单元数量
rnn_units = 1024
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, return_sequences=True)),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
  ])
    
    
#   model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(vocab_size, embedding_dim,
#                               batch_input_shape=[batch_size, None]),
#     tf.keras.layers.GRU(rnn_units,
#                         return_sequences=True,
#                         stateful=True,
#                         recurrent_initializer='glorot_uniform'),
#     tf.keras.layers.Dense(vocab_size)
#   ])
  return model

In [18]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 100, 3173) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 256)          812288    
_________________________________________________________________
bidirectional (Bidirectional (128, None, 2048)         10493952  
_________________________________________________________________
bidirectional_1 (Bidirection (128, None, 2048)         25174016  
_________________________________________________________________
dense (Dense)                (128, None, 3173)         6501477   
Total params: 42,981,733
Trainable params: 42,981,733
Non-trainable params: 0
_________________________________________________________________


In [21]:
import time,datetime
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
# 检查点保存至的目录
checkpoint_dir = './training_checkpoints_jieba'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_"+str(time.time()))

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    save_freq='epoch')

In [ ]:
EPOCHS=1
# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 1 steps


In [ ]:
# latest = tf.train.latest_checkpoint(checkpoint_dir)
# model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# model.load_weights(latest)
# model.build(tf.TensorShape([1, None]))
model.summary()

In [ ]:
def generate_text(model, start_string):
  # 评估步骤（用学习过的模型生成文本）

  # 要生成的字符个数
  num_generate = 1000
  num_sentences = 6
  input_str = jp.cut(start_string,use_paddle=True)
  input_eval = []
  for s in input_str:
    print(char2idx[s.word])
    input_eval.append(char2idx[s.word])
  # 将起始字符串转换为数字（向量化）
#   input_eval = [char2idx[s.word] for s in input_str]
  print(input_eval)
  input_eval = tf.expand_dims(input_eval, 0)
  print(input_eval)
  # 空字符串用于存储结果
  text_generated = []

  # 低温度会生成更可预测的文本
  # 较高温度会生成更令人惊讶的文本
  # 可以通过试验以找到最好的设定
  temperature = 1.0

  # 这里批大小为 1
  model.reset_states()
  i = 0
  while True:
    if i == num_sentences:
      break
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)
    if idx2char[predicted_id] != 'END':
      text_generated.append(idx2char[predicted_id])
    else:
      text_generated.append(idx2char[predicted_id])
      i += 1
    if idx2char[predicted_id] == '\n':
      print('end')
  return (start_string + ''.join(text_generated))
print(generate_text(model, start_string='人民广场'))